### Make imports

In [ ]:
import io
import os
import sys
import types
import argparse
import re
import requests

from urllib.parse import urlparse

from IPython import get_ipython
from IPython.core.magic import register_cell_magic
from IPython.core.interactiveshell import InteractiveShell
from io import StringIO
from nbformat import read

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

### Define constants

In [ ]:
ACCEPTED_FILE_EXT = ['ipynb', 'csv', 'py', 'txt']

### Define parser

In [ ]:
def str2bool(v):
    if isinstance(v, bool):
        return v
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

parser = argparse.ArgumentParser()
parser.add_argument('--alias', type = str, required = False)
parser.add_argument('--override', default=False, type = str2bool, required = False)

### Code to make ipynb import
https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html

In [ ]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path


class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)

        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path='.'):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]


### Download the file to current container

In [ ]:
def download(dependency_url, alias=None, file_type='ipynb', auth=None, verify=False, **kwargs):
    parse_path = urlparse(dependency_url)
    path = "/".join(parse_path.path.rsplit('/')[2:])
    netloc = parse_path.netloc
    scheme = parse_path.scheme
    content_api = f"{scheme}://{netloc}/api/contents/"
    final_path = content_api + path
    res = requests.get(final_path, auth=auth, verify=verify)
    with open(alias, 'w') as out:
        if file_type == 'ipynb':
            data = json.dumps(res.json()['content'])
        elif file_type in ['csv', 'py']:
            data = res.json()['content']
        else:
            raise Exception(f"Illigal file type : {file_type}. Currently it supports only .ipynb, .csv, .py and .txt")
        out.write(data)

### Parse the cell containent

In [ ]:
def parse_cell(cell_info):
    load_nbloader = False
    buf = cell_info.getvalue().strip().split('\n')
    for line in buf:

        line = line.split()
        file_path = line[0]
        file_ext = file_path.split('.')[-1]
        if file_ext not in ACCEPTED_FILE_EXT:
            raise Exception(f"Illigal file type : {file_ext}. Currently it supports only .ipynb, .csv, .py and .txt")
        
        args = parser.parse_args(line[1:])
        params = vars(args)
        alias = file_path.split('/')[-1] if not params['alias'] else params['alias']    
        
        is_file_name_exists = os.path.isfile(alias)

        if is_file_name_exists and not params['override']:
            print(f"Skipping file with name {alias} since it already exists and override is False.")
        else:
            if is_file_name_exists:
                print(f"Loading {file_path} and overriding the existing file with name {alias}")
            else:
                print(f"Loading {file_path} and saving it as {alias}")
        
        download(file_path, alias, file_type=file_ext)
        if file_ext == 'ipynb':
            load_nbloader = True

    return load_nbloader

### Magic for loading

In [ ]:
@register_cell_magic
def load_dependencies(line, cell):
    sio = StringIO(cell)
    if sio:
        load_nbloader = parse_cell(sio)
        if load_nbloader:
            sys.meta_path.append(NotebookFinder())

### Test load dependencies magic

In [ ]:
%%load_dependencies


In [ ]:
!ls